In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql.types import IntegerType

In [17]:
student_data = [
    {"student_id": 1,"name":"Khan"},
     {"student_id": 2,"name":"Rahul"},
    {"student_id": 3,"name":"Lavan"},
]

In [3]:
spark = SparkSession.builder.master("local[2]").appName("StudentMarks").getOrCreate()

In [18]:
student = spark.createDataFrame(student_data)
student.show()

+-----+----------+
| name|student_id|
+-----+----------+
| Khan|         1|
|Rahul|         2|
|Lavan|         3|
+-----+----------+



In [20]:
student_marks = [
    {"student_id": 1,"subject":"Pyspark","marks":90},
    {"student_id": 1,"subject":"SQL","marks":100},
    {"student_id": 2,"subject":"SQL","marks":70},
    {"student_id": 2,"subject":"Pyspark","marks":60},
    {"student_id": 3,"subject":"SQL","marks":30},
    {"student_id": 3,"subject":"Pyspark","marks":20},
]

In [21]:
student_marks = spark.createDataFrame(student_marks)
student_marks.show()

+-----+----------+-------+
|marks|student_id|subject|
+-----+----------+-------+
|   90|         1|Pyspark|
|  100|         1|    SQL|
|   70|         2|    SQL|
|   60|         2|Pyspark|
|   30|         3|    SQL|
|   20|         3|Pyspark|
+-----+----------+-------+



### Calculate the % Marks for each student.
**Each subject is of 100 marks.**
Create the Division column by following below condition.
<ol>
<li> % Marks greater than or equal to 70 then "Distinction"</li>
<li> % Marks range between 60-69 then "First class"</li>
<li> % Marks range between 50-59 then "Second class"</li>
<li> % Marks range between 40-49 then "Third class"</li>
<li> % Marks less then or equal to 39 the "fail"</li>

In [23]:
df = student.join(student_marks, student.student_id == student_marks.student_id,"inner").\
groupBy(student.student_id,student.name).agg(FS.sum('marks').alias("total")).\
withColumn("percentage",FS.expr('(total * 100)/200')).\
withColumn("Division",
           FS.when(FS.col('percentage') >= 70,"Distinct").\
           when(FS.col('percentage').between(60, 69),"First class").\
           when(FS.col('percentage').between(50, 59),"Second class").\
           when(FS.col('percentage').between(40, 49),"Third class").\
           otherwise("Fail")
          ).drop("total")

In [24]:
df.select("*").show()

+----------+-----+----------+-----------+
|student_id| name|percentage|   Division|
+----------+-----+----------+-----------+
|         1| Khan|      95.0|   Distinct|
|         2|Rahul|      65.0|First class|
|         3|Lavan|      25.0|       Fail|
+----------+-----+----------+-----------+

